## Running AI Red Teaming Agent ##
- This item is in preview so with that being said this code and syntax can likely change once this goes into "GA"
- Region Support (East US 2, Sweden Central, France Central, Switzerland West)
- Ensure you authenticate via CLI when launching this (az login --use-device-code)
- For the evaluation upload if you run into issues ensure Credential (assigned to user is Storage Blob Data Contributor to associated storage account or check network settings if segmented can be a challenge)
- This specific use of this tool should be used with Python 3.11, 3.12 - for some reason PyRiT isn't able to run on Python 3.13 this is highlighted in the documentation but don't want you to miss the nuance

In [ ]:
%pip install azure-ai-evaluation[redteam]
%pip install python-dotenv
%pip install azure-ai-projects
%pip install azure-identity
%pip install pyrit==0.9.0
%pip install openai==1.99.9

In [ ]:
# Azure imports
import os
from dotenv import load_dotenv
from azure.identity import DefaultAzureCredential, AzureCliCredential, get_bearer_token_provider
from azure.ai.evaluation.red_team import RedTeam, RiskCategory

credential = AzureCliCredential()
# Load environment variables from .env file
load_dotenv()

azure_subscription_id = os.environ.get("AZURE_SUBSCRIPTION_ID")
azure_resource_group_name = os.environ.get("AZURE_RESOURCE_GROUP_NAME")
azure_ai_project_name = os.environ.get("AZURE_PROJECT_NAME_HUB")
azure_openai_api_version = os.environ.get("AZURE_API_VERSION")

if not azure_subscription_id:
    raise ValueError("AZURE_SUBSCRIPTION_ID environment variable is not set.")

azure_ai_project = os.environ.get("AZURE_PROJECT_ENDPOINT")

if not azure_ai_project:
    raise ValueError("AZURE_PROJECT_ENDPOINT environment variable is not set.")

# Azure AI Project Information
azure_ai_project = {
    "subscription_id": azure_subscription_id,
    "resource_group_name": azure_resource_group_name,
    "project_name": azure_ai_project_name,
}


# Instantiate your AI Red Teaming Agent
red_team_agent = RedTeam(
    azure_ai_project=azure_ai_project, # required
    credential=credential, # required
    num_objectives=5, # optional, defaults to 10
)

## Creating a target ##
A target represents in this context the LLM we are evaluating from a red-team perspective.

In [ ]:
# Configuration for your Azure OpenAI Model
# ---> This will be gpt-4o or gpt-4o-mini, etc..
from dotenv import load_dotenv
import os

# Load .env file
load_dotenv()

azure_endpoint = os.environ.get("AZURE_OPENAI_ENDPOINT")
azure_api_key = os.environ.get("AZURE_OPENAI_KEY")
azure_deployment_name = os.environ.get("AZURE_OPENAI_DEPLOYMENT_NAME")
azure_api_version = os.environ.get("AZURE_API_VERSION")

# Validation that this is set
if not azure_endpoint or not azure_deployment_name:
    raise ValueError("Please set AZURE_OPENAI_ENDPOINT and AZURE_OPENAI_DEPLOYMENT_NAME in your .env file.")

# configuration for Azure OpenAI
azure_openai_config = {
    "azure_endpoint": azure_endpoint, # not required if authenticated with az login --use-device-code
    "api_key": azure_api_key, # not required if authenticated with az login --use-device-code
    "azure_deployment": azure_deployment_name, # IMPORTANT if running AzureOpenAI explicitly input that azure_deployment as parameter
}

# Use of Attack Strategy #
This uses the grouping of the class 'AttackStrategy' for various stages the comments are here to explain

In [ ]:
from azure.ai.evaluation.red_team import AttackStrategy


# Run the red team scan with multiple attack strategies
advanced_result = await red_team_agent.scan(
    target=azure_openai_config,
    scan_name="July-Scan",
    attack_strategies=[
        AttackStrategy.EASY,  # Group of easy complexity attacks
        AttackStrategy.MODERATE,  # Group of moderate complexity attacks
        AttackStrategy.CharacterSpace,  # Add character spaces
        AttackStrategy.ROT13,  # Use ROT13 encoding
        AttackStrategy.UnicodeConfusable,  # Use confusable Unicode characters
        AttackStrategy.CharSwap,  # Swap characters in prompts 
        AttackStrategy.Morse,  # Encode prompts in Morse code - ".---..---"
        AttackStrategy.Leetspeak,  # Use Leetspeak think of 1337 h4x0r
        AttackStrategy.Url,  # Use URLs in prompts "r0b%20r0b%20r0b%20r0b the $bA3b3$",
        AttackStrategy.Binary,  # Encode prompts in binary
        AttackStrategy.Compose([AttackStrategy.Base64, AttackStrategy.ROT13]),  # Use two strategies in one attack
    ],
    output_path="Scan.json", # if you want to define the .json file output path
)

## Alternative use with custom prompts ##

In [12]:
from azure.identity import AzureCliCredential, get_bearer_token_provider

path_to_prompts = "./data/prompts.json" # Path to your custom prompts file

credential = AzureCliCredential()  # Use Azure CLI credentials for authentication


# Run the red team scan with custom prompts
red_team_custom = RedTeam(
    azure_ai_project=azure_ai_project, # required
    credential=credential, # required
    custom_attack_seed_prompts=path_to_prompts, # optional, defaults to None (no custom prompts
)



## Send the attack using the .scan() ##
- This shows after initialization of red_team_custom we send our attack.

In [ ]:
custom_scan_result = await red_team_custom.scan(
    target=azure_openai_config,
    scan_name="Custom-Prompts-August",
    output_path="NewScan.json",
    skip_upload=True,  # if you want to define the .json file output path
)

In [ ]:
from pyrit.prompt_target import OpenAIChatTarget, PromptChatTarget
from azure.ai.evaluation.red_team import RedTeam
from azure.identity import AzureCliCredential, DefaultAzureCredential

# Default
credential = DefaultAzureCredential()

chat_target = OpenAIChatTarget(
    model_name=azure_deployment_name,
    endpoint=azure_endpoint,
    api_key=azure_api_key,  # not required if authenticated with az login --use-device-code
)



red_team_scan = RedTeam(
    azure_ai_project=azure_ai_project,
    credential=credential,
    application_scenario="chat",
    num_objectives=5,
)

results = await red_team_scan.scan(
    target=chat_target,
    scan_name="Red Team Scan with Custom Target",
    output_path="AugustScan.json",
    skip_upload=True
)
